## Accessing USDA Food Data Central API for Nutritional Information

This script interacts with the USDA Food Data Central API to search for food items and retrieve nutritional information.

In [2]:
#!pip install requests
import requests

# Define your API key (it should be the actual key, not "OAS3.0:")
api_key = 'M3175TAXsEcfdKkCOpNO9VcbwjQtFOkl9rUDAiaP'

# Function to search for a food item and get its FDC ID
def search_food(query):
    #This is the data set for the USDA government
    url = "https://api.nal.usda.gov/fdc/v1/foods/search"
    # my parameters are query for the 
    params = {
        "query": query, # search term enterd by user 
        "api_key": api_key, 
        "pageSize": 1  # Return only the top result
    }
    # using request.get() to send a get request to the API endpoint
    response = requests.get(url, params=params)
    #lines checks for the Https status and if it 200 then its okay
    if response.status_code == 200:
        # returns the json format to dictionary
        data = response.json()
        #checks for food items 
        if data['foods']:
            # this will get the first result 
            food_item = data['foods'][0]
            # return the food id and its description
            return food_item['fdcId'], food_item['description']
        else:
           #No foods were found
            print("No foods found.")
            return None, None
    else:
      # it will print out the error responce code
        print(f"Error: {response.status_code}")
        return None, None

# Function to retrieve detailed nutritional information using FDC ID
def get_food_details(fdc_id):

    url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}"
    params = {
        "api_key": api_key,
        "format": "full"#All available details for food
    }
    # Returns the full detail information for the food 
    response = requests.get(url, params=params)
    # Same thing as earlier 
    if response.status_code == 200:
        food_details = response.json()
        return food_details
    else:
        print(f"Error: {response.status_code}")
        return None

# whatever the user wants
foodname = "apple"
# Example usage
if __name__ == "__main__":
    food_query = foodname
    fdc_id, description = search_food(food_query)
    
    if fdc_id:
        print(f"FDC ID: {fdc_id}, Description: {description}")
        food_details = get_food_details(fdc_id)
        
        # Print out some of the nutritional information
        if food_details:
            print(f"\nNutritional Information for {description}:")
            for nutrient in food_details.get('foodNutrients', []):
                print(f"{nutrient['nutrient']['name']}: {nutrient['amount']} {nutrient['nutrient']['unitName']}")
    else:
        print("Could not retrieve food details.")

FDC ID: 454004, Description: APPLE

Nutritional Information for APPLE:
Fatty acids, total saturated: 0.0 g
Calcium, Ca: 0.0 mg
Vitamin C, total ascorbic acid: 3.1 mg
Cholesterol: 0.0 mg
Iron, Fe: 0.23 mg
Sodium, Na: 0.0 mg
Vitamin A, IU: 65.0 IU
Total Sugars: 10.39 g
Energy: 52.0 kcal
Potassium, K: 110.0 mg
Protein: 0.0 g
Fatty acids, total trans: 0.0 g
Fiber, total dietary: 3.2 g
Carbohydrate, by difference: 14.29 g
Total lipid (fat): 0.65 g


## Extract Video Frames in Smaller Batches

This script extracts frames from a video file and saves them in smaller batches within a designated folder.


In [3]:
import cv2
import os

def video_to_frames_in_smaller_batches(video_file, original_folder, batch_size=300):
    # Create a new folder inside the original folder based on the video file name
    video_filename = os.path.basename(video_file)
    video_name, _ = os.path.splitext(video_filename)
    output_folder = os.path.join(original_folder, video_name)

    # Create the output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(video_file)
    frame_count = 0
    batch_count = 0
    batch_folder = os.path.join(output_folder, f'batch_{batch_count:03d}')
    os.makedirs(batch_folder, exist_ok=True)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save the frame to the batch folder
        frame_filename = os.path.join(batch_folder, f'frame_{frame_count:05d}.jpg')
        cv2.imwrite(frame_filename, frame)
        frame_count += 1

        # Create a new batch folder after every batch_size frames
        if frame_count % batch_size == 0:
            batch_count += 1
            batch_folder = os.path.join(output_folder, f'batch_{batch_count:03d}')
            os.makedirs(batch_folder, exist_ok=True)

    cap.release()
    print(f"Extracted {frame_count} frames into {batch_count + 1} batches in the folder '{output_folder}'.")

# Example usage with a writable directory:
video_file = "/Users/taneshafuller/Desktop/FinalProductDataset/"  # Replace with your video file path
original_folder = os.path.expanduser("~/Documents/ProjectRene/FinalDataset/CutTuna2images")  # Change to a writable location
video_to_frames_in_smaller_batches(video_file, original_folder)

Extracted 205 frames into 1 batches in the folder '/Users/taneshafuller/Documents/ProjectRene/FinalDataset/CutTuna2images/5900834-uhd_2160_4096_25fps'.


## Video segmentation with SAM 2
This notebook shows how to use SAM 2 for interactive segmentation in videos. It will cover the following:

- adding clicks (or box) on a frame to get and refine _masklets_ (spatio-temporal masks)
- propagating clicks (or box) to get _masklets_ throughout the video
- segmenting and tracking multiple objects at the same time

We use the terms _segment_ or _mask_ to refer to the model prediction for an object on a single frame, and _masklet_ to refer to the spatio-temporal masks across the entire video. 

## Annotating Images with Bounding Boxes for Object Detection

This script helps annotate objects in images by drawing bounding boxes and saving them as training labels for models.
If running locally using jupyter, first install `segment-anything-2` in your environment using the [installation instructions](https://github.com/facebookresearch/segment-anything-2#installation) in the repository.


## Set env for SAM2 

In [2]:
using_colab = False
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything-2.git'

    !mkdir -p videos
    !wget -P videos https://dl.fbaipublicfiles.com/segment_anything_2/assets/bedroom.zip
    !unzip -d videos videos/bedroom.zip

    !mkdir -p ../checkpoints/
    !wget -P ../checkpoints/ https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt

    

In [3]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image

In [4]:
# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

using device: mps

Support for MPS devices is preliminary. SAM 2 is trained with CUDA and might give numerically different outputs and sometimes degraded performance on MPS. See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion.


## Loading SAM2 Video Predictor

In [7]:
from sam2.build_sam import build_sam2_video_predictor

sam2_checkpoint = "../checkpoints/sam2_hiera_large.pt"
model_cfg = "sam2_hiera_l.yaml"

predictor = build_sam2_video_predictor(model_cfg, sam2_checkpoint, device=device)

FileNotFoundError: [Errno 2] No such file or directory: '../checkpoints/sam2_hiera_large.pt'